In [1]:
import os
from pygame import mixer
from gtts import gTTS
my_file = open("speach.txt","r")
my_text = my_file.read()
my_file.close()
# import pyaudio
# import speech_recognition as sr


pygame 2.1.2 (SDL 2.0.18, Python 3.8.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


win+r
cmd
pip install py

In [9]:
pip --version

pip 23.0 from /Users/pattycha/Library/Python/3.8/lib/python/site-packages/pip (python 3.8)
Note: you may need to restart the kernel to use updated packages.
